In [2]:
%load_ext autoreload

%autoreload 2

In [32]:
import datajoint as dj

dj.config["database.host"] = "at-database.ad.bcm.edu"

from neuropixel_pipeline.readers import Adapters
from neuropixel_pipeline.readers.labview import LabviewNeuropixelMeta
from neuropixel_pipeline.schemata import base, probe, ephys

from pathlib import Path

In [24]:
base.Session.heading

# Session: table connection
session_id           : int unsigned                 # Session primary key

In [27]:
# Uses contents only, lookup table
ephys.AcquisitionSoftware()

acq_software
LabviewSaumil
Open Ephys
SpikeGLX


In [29]:
# 
# ephys.ProbeInsertion.insert1()

session_dir = Path('')
labview_metadata = LabviewNeuropixelMeta.from_h5(session_dir)
ephys.ProbeInsertion().heading

# Probe insertion implanted into an animal for a given session.
session_id           : int unsigned                 # Session primary key
insertion_number     : tinyint unsigned             # 
---
probe                : varchar(32)                  # unique identifier for this model of probe (e.g. serial number)

In [38]:
key = {'animal_id': 29187,
      'session': 25,
      'scan_idx': 1}

experiment = dj.create_virtual_module('experiment', 'pipeline_experiment')

session_rel = experiment.Session() & key
session_rel

animal_id id number,session session index for the mouse,rig multiphoton imaging setup,session_date date,username lab member,anesthesia anesthesia short name,scan_path file path for TIFF stacks,"behavior_path pupil movies, whisking, locomotion, etc.",craniotomy_notes free-text notes,session_notes free-text notes,archive archive disk label,session_ts automatic
29187,25,RS1,2023-05-16,maria,awake,W:\Two-Photon\maria\2023-05-16_13-39-49,W:\Two-Photon\maria\2023-05-16_13-39-49,,,RS1A2A,2023-05-16 13:39:49


In [39]:
session_rel.fetch1()

{'animal_id': 29187,
 'session': 25,
 'rig': 'RS1',
 'session_date': datetime.date(2023, 5, 16),
 'username': 'maria',
 'anesthesia': 'awake',
 'scan_path': 'W:\\Two-Photon\\maria\\2023-05-16_13-39-49',
 'behavior_path': 'W:\\Two-Photon\\maria\\2023-05-16_13-39-49',
 'craniotomy_notes': ' ',
 'session_notes': ' ',
 'archive': 'RS1A2A',
 'session_ts': datetime.datetime(2023, 5, 16, 13, 39, 49)}

In [25]:
ephys.ProbeInsertion.heading

# Probe insertion implanted into an animal for a given session.
session_id           : int unsigned                 # Session primary key
insertion_number     : tinyint unsigned             # 
---
probe                : varchar(32)                  # unique identifier for this model of probe (e.g. serial number)